In [218]:
%run -i header.py
from sympy import N
from sympy.physics.quantum.cg import CG
from scipy.special import sph_harm, spherical_jn
from cubature import cubature

Define clebsh

$<abcd|ef>$

In [219]:
def clebsch(a, b, c, d, e ,f):
    return sp.array(N(CG(a, b, c, d, e, f).doit()), dtype=sp.complex64)

Define polarization vectors:

- $\lambda = \pm1:\qquad \vec{e}_\lambda = -\frac{\lambda}{\sqrt{2}} \{ 1, \mathrm{i} \lambda, 0 \}$
- $\lambda = 0:\qquad \vec{e}_\lambda = \{ 0, 0, 1 \}$

In [220]:
def ee(lam):
    return -lam/sp.sqrt(2)*sp.array([1, lam*1j, 0], dtype=sp.complex64) + sp.array([0, 0, 1-lam**2])

Define vector spherical harmonic:

$\vec{Y}_{L, J}^\lambda = \sum_{m_1, m_2} <J \lambda|l m_1 1 m_2> Y_L^{m_1} \vec{e}_{m_2}$

In [221]:
def Yvec(l, J, m, th, phi):
    res = 0
    for m1 in range(-l, l+1, 1):
        for m2 in range(-1, 2, 1): 
            res += sph_harm(m1, l, phi, th)*clebsch(l, m1, 1, m2, J, m)*ee(m2)
    return  res

Spherical harmonic of the order 0 doesn't depend on $\phi$ in general notation. It is represented in notation of SciPy `sph_harm`.

Adaptation scheme: $Y_l^m(\theta, \phi)$ = `sph_harm`$(m, l, \phi, \theta)$

In [222]:
sph_harm(0, 5, 3.2, 0.2) - sph_harm(0, 5, 2.4, 0.2)

0j

In [235]:
def sph_harm(m, l, phi, th):
    res = sp.special.sph_harm(m, l, phi, th)
    return sp.nan_to_num(res)

Plane wave expansion according to [Wiki](https://en.wikipedia.org/wiki/Plane_wave_expansion).

We apply the following property of spherical harmonics:

$Y_l^0(\theta, \phi) = \sqrt{\frac{2l+1}{4 \pi}} P_l(\cos(\theta))$

In [236]:
def testLegendreSph(n, th):
    return sph_harm(0, n, 0, th) - sp.sqrt((2*n+1)/4/sp.pi)*sp.special.legendre(n)(sp.cos(th))
testLegendreSph(4,0.2)

(-3.3306690738754696e-16+0j)

In [237]:
def pwExp(l, kr, th):
    return sp.sqrt(4*sp.pi*(2*l+1))*1j**l*spherical_jn(l, kr)*sph_harm(0, l, 0, th)
def pwExpSum(lmax, kr, th):
    return pwExp(sp.arange(0, lmax, 1), kr, th).sum()

In [238]:
pwExpSum(10, 1.3, 0.2) - sp.exp(1j*1.3*sp.cos(0.2))

(3.3868731708075472e-09+7.025391379755774e-11j)

In [239]:
def testClebschOrtho(j1, j2, M1, M2):
    return sum([ clebsch(j1, M1[0], j2, M1[1], j, sum(M1))*clebsch(j1, M2[0], j2, M2[1], j, sum(M2)) for j in range(abs(j1-j2), (j1+j2) +1, 1)])
testClebschOrtho(1, 2, (1,2), (1,3))

0j

In [240]:
def testOrtho(J, lam, th):
    return sph_harm(0, J, 0, th)*ee(lam) - sum([Yvec(J, j, lam, th, 0)*clebsch(J, 0, 1, lam, j, lam) for j in range(abs(J-1), J+2, 1)])
testOrtho(2, 1, 0.3)

array([-9.64022001e-09+0.00000000e+00j,  0.00000000e+00-8.83757939e-09j,
       -4.75679567e-09+0.00000000e+00j])

In [241]:
def prepremult(jmax, lam, kr, th):
    return sum([\
         1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J,lam)*Yvec(J,J,lam,th,0)\
        for J in range(1, jmax+1, 1)])\
    +sum([\
         1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J+1,lam)*Yvec(J,J+1,lam,th,0)\
        for J in range(0, jmax+1, 1)])\
    +sum([\
         1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J-1,lam)*Yvec(J,J-1,lam,th,0)\
        for J in range(1, jmax+1, 1)])
display(prepremult(10, 1, 1, 0.2) - ee(1)*sp.exp(1j*sp.cos(0.2)))

array([-4.92803814e-09+1.97833974e-08j, -1.98446799e-08-4.73346518e-09j,
       -1.57252362e-09-5.00387396e-11j])

In [242]:
def premult(jmax, lam, kr, th):
    return sum([\
         1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J,lam)*Yvec(J,J,lam,th,0)\
        for J in range(1, jmax+1, 1)])\
    +sum([\
         -1j**(J+1)*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J-1, kr)*clebsch(J-1,0,1,lam,J,lam)*Yvec(J-1,J,lam,th,0)*sp.sqrt((2*J-1)/(2*J+1))\
        for J in range(1, jmax+1, 1)])\
    +sum([\
         1j**(J+1)*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J+1, kr)*clebsch(J+1,0,1,lam,J,lam)*Yvec(J+1,J,lam,th,0)*sp.sqrt((2*J+3)/(2*J+1))\
        for J in range(0, jmax+1, 1)])
display(premult(15, 1, 1, 0.2) - ee(1)*sp.exp(1j*sp.cos(0.2)))

array([-4.92793162e-09+1.97861991e-08j, -1.98474815e-08-4.73335865e-09j,
       -1.57252361e-09-5.00387553e-11j])

In [243]:
def mult(J, lam, kr, th):
    return -sp.sqrt(2*sp.pi)*(1j)**J*sp.sqrt(2*J+1)*(lam*spherical_jn(J, kr)*Yvec(J, J, lam, th, 0.)+1j*(sp.sqrt((J+1)/(2*J+1))*spherical_jn(J-1, kr)*Yvec(J-1, J, lam, th, 0) - sp.sqrt(J/(2*J+1))*spherical_jn(J+1, kr)*Yvec(J+1, J, lam, th, 0)))

In [244]:
sum([mult(J, 1, 1, 0.2) for J in range(1,15)]) - ee(1)*sp.exp(1j*1*sp.cos(0.2))

array([-2.49798704e-09+9.87284032e-09j, -9.94399985e-09-2.33265757e-09j,
       -1.84763736e-09+4.23745651e-11j])

Multipole expansion:

$ \vec{\epsilon}_\lambda \mathrm{e}^{-\mathrm{i} \xi k r \cos{\theta}} = -\sqrt{2 \pi} \sum_{J = 1}^{\infty} (-\mathrm{i} \xi)^J \sqrt{2J + 1}\left[\lambda j_J(kr) \vec{Y}_{J, J}^{\lambda} - \mathrm{i} \xi \left(\sqrt{\frac{J+1}{2J+1}}j_{J-1}(kr) \vec{Y}_{J-1, J}^\lambda - \sqrt{\frac{J}{2J+1}}j_{J+1}(k r) \vec{Y}_{J+1,J}^\lambda \right)\right] $

In [245]:
def mult(J, lam, kr, th, xi=1):
    return -sp.sqrt(2*sp.pi)*(-1j*xi)**J*sp.sqrt(2*J+1)*(lam*spherical_jn(J, kr)*Yvec(J, J, lam, th, 0.)-1j*xi*(sp.sqrt((J+1)/(2*J+1))*spherical_jn(J-1, kr)*Yvec(J-1, J, lam, th, 0) - sp.sqrt(J/(2*J+1))*spherical_jn(J+1, kr)*Yvec(J+1, J, lam, th, 0)))
sum([mult(J, 1, 1, 0.2, -1) for J in range(1,15)]) - ee(1)*sp.exp(1j*1*sp.cos(0.2))

array([-2.49798704e-09+9.87284032e-09j, -9.94399985e-09-2.33265757e-09j,
       -1.84763736e-09+4.23745651e-11j])

Result: Multipole expansion works as in the thesis

## Coeff Q symmetry

In [18]:
def coefQ(J, lam, instate, outstate):
    return sum([clebsch(outstate["L"], outstate["m"]-s, outstate["S"], s, outstate["J"], outstate["m"])*\
         clebsch(instate["L"], instate["m"]-s, instate["S"], s, instate["J"], instate["m"])*\
         clebsch(J, -lam, instate["L"], instate["m"]-s, outstate["L"], outstate["m"]-s)
        for s in range(-instate["S"], instate["S"]+1, 1)])**2
    
a1 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})# chi_c2 -> psi_1S
a2 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1}) #chi_c2 -> psi_1S
display(a1/a2)

b1 = coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":1, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":-1, "L":1, "S":1})# psi_3S -> chi_c2
b2 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":2, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":-1, "L":0, "S":1}, {"J":2, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":-1, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":1, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":2, "m":-2, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":1, "L":0, "S":1}, {"J":2, "m":2, "L":1, "S":1})
display(b1/b2)

a1 = coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})# chi_c2 -> psi_1S
a2 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1}) #chi_c2 -> psi_1S
display(a1/a2)


(0.10000000506295846+0j)

(0.29999999062850013+0j)

(0.6000000040888882+0j)

In [19]:
b1 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})# psi_3S -> chi_c2
b2 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})
display(b1/b2)

(0.5+0j)

# Test $H_{int}$

## Raw

L-average

$$ <L_f l_f| f(\theta, \phi) \mathrm{e}^{\mathrm{i} k r \cos{\theta}} |L_i l_i> = \\
= \int\sin{\theta}d\theta d\phi~Y_{L_i}^{\star l_i} f(\theta, \phi) \mathrm{e}^{\mathrm{i} k r \cos{\theta}} Y_{L_f}^{l_f} $$

In [275]:
def LexpMel(ins, outs, k, r, f=lambda th, ph: 1):
    def spOrth_f(x, k, r, ins, outs, f):
        return sp.array([sp.real(sp.sin(x.T[1])*f(x.T[1], x.T[0])*sp.exp(1j*k*r*sp.cos(x.T[1]))*sp.conj(sph_harm(outs["l"], outs["L"], *x.T))*sph_harm(ins["l"], ins["L"], *x.T)),
                  sp.imag(sp.sin(x.T[1])*f(x.T[1], x.T[0])*sp.exp(1j*k*r*sp.cos(x.T[1]))*sp.conj(sph_harm(outs["l"], outs["L"], *x.T))*sph_harm(ins["l"], ins["L"], *x.T))]).T
    res = cubature(spOrth_f, 2, 2, sp.array([0, 0]), sp.array([2*sp.pi, sp.pi]), kwargs={"ins": ins, "outs": outs, "k": k, "r": r, "f": f}, vectorized=True)[0]
    return res[0] + 1j*res[1]
    

Spinless Average

$$
    <J_f j_f L_f S_f| \vec{r} \vec{\epsilon}^{\star}_{\lambda} \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}} |J_i j_i L_i S_i> = \\
    = -\sum_s r \sqrt{\frac{4 \pi}{3}} <L_f (j_f - s) S_f s | J_f j_f> \times \nonumber \\
      \times <L_i (j_i - s) S_i s | J_i j_i> \times \nonumber \\
      \times <L_f (j_f-s)| Y_{1}^{-\lambda} \mathrm{e}^{\mathrm{i} k\frac{r}{2} \cos{\theta}} |L_i (j_i - s) >
$$

In [529]:
def spinlessAv(ins, outs, k, r, lam):
    if round(2*ins["S"]+1) != round(2*outs["S"]+1):
        return 0
    res = 0 
    for s in range(-ins["S"], ins["S"]+1):
        res += clebsch(outs["L"], outs["j"]-s, outs["S"], s, outs["J"], outs["j"])\
             * clebsch(ins["L"], ins["j"]-s, ins["S"], s, ins["J"], ins["j"])\
             * LexpMel({"L": ins["L"], "l": ins["j"] - s}, {"L": outs["L"], "l": outs["j"] - s}, k, r/2, f=lambda th, phi: sph_harm(-lam, 1, phi, th))
    return -r*sp.sqrt(4*sp.pi/3)*res

Average of a spin-containing part

$$  <J_f j_f L_f S_f| \vec{\sigma}_{q} \vec{\epsilon}^{\star}_{\lambda} \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}} |J_i j_i L_i S_i > = -\sum_{s, s_q, s_\overline{q}} \sqrt{2} \lambda <L_f (j_f - (s_q+s_{\overline{q}} - \lambda) S_f (s_q+s_{\overline{q}} - \lambda) | J_f j_f> \times \nonumber \\
    <L_i (j_i - (s_q+s_{\overline{q}}) S_i (s_q+s_{\overline{q}}) | J_i j_i> \times \nonumber \\
    <\frac{1}{2} s_q \frac{1}{2} s_{\overline{q}} | S_i (s_q + s_{\overline{q}})> \times \nonumber \\
    <\frac{1}{2} (s_q - \lambda) \frac{1}{2} s_{\overline{q}} | S_f (s_q + s_{\overline{q}} - \lambda)> \times \nonumber \\
    <L_f (j_f - (s_q + s_{\overline{q}} - \lambda))| \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}} |L_i (s_q + s_{\overline{q}})> 
$$

In [530]:
def spinAv(ins, outs, k, r, lam):
    res = 0 
    for sq in (-1/2, 1/2):
        for sqbar in (-1/2, 1/2):
            res += clebsch(outs["L"], outs["j"]-(sq+sqbar-lam), outs["S"], sq+sqbar-lam, outs["J"], outs["j"])\
                 * clebsch(ins["L"], ins["j"]-(sq+sqbar), ins["S"], sq+sqbar, ins["J"], ins["j"])\
                 * clebsch(1/2, sq, 1/2, sqbar, ins["S"], sq+sqbar)\
                 * clebsch(1/2, sq-lam, 1/2, sqbar, outs["S"], sq+sqbar-lam)\
                 * LexpMel({"L": ins["L"], "l": ins["j"] - (sq+sqbar)}, {"L": outs["L"], "l": outs["j"] - (sq+sqbar-lam)}, k, r/2)
    return -sp.sqrt(2)*lam*res

Raw version of $H_{int}$:

$$ H_{int} = -\frac{\mathrm{i}}{2} (E_f - E_i) \left(<J_f j_f L_f S_f| \vec{r} \vec{\epsilon}^\star_{\lambda} \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}}|J_i j_i L_i S_i> + <J_f j_f L_f S_f| \vec{r} \vec{\epsilon}^\star_{\lambda} \mathrm{e}^{\mathrm{i} (-k) \frac{r}{2} \cos{\theta}}) |J_i j_i L_i S_i> \right) - \nonumber \\
- \frac{\lambda k}{4 \mu} \left( <J_f j_f L_f S_f| \vec{\sigma}_q \vec{\epsilon}^\star_{\lambda} \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}} |J_i j_i L_i S_i> - (-1)^{S_f + S_i} <J_f j_f L_f S_f| \vec{\sigma}_q \vec{\epsilon}^\star_{\lambda} \mathrm{e}^{\mathrm{i} (-k) \frac{r}{2} \cos{\theta}} |J_i j_i L_i S_i> \right) $$

In [307]:
def Hint_raw(ins, outs, k, r, lam):
    mu = 1
    return -1j/2*(outs["E"]-ins["E"])*(spinlessAv(ins, outs, k, r, lam)+spinlessAv(ins, outs, -k, r, lam))-lam*k*1/4/mu*(spinAv(ins, outs, k, r, lam) - (-1)**round(ins["S"] + outs["S"])*spinAv(ins, outs, -k, r, lam))

In [308]:
Hint_raw({"J": 1, "j": 1, "L": 0, "S": 1, "E": 1}, {"J": 1, "j": 0, "L": 0, "S": 1, "E": 0}, 1, 1, 1)

(-0+6.071532062008978e-18j)

## Thesis

$$ C_{L, J}^{\lambda} = \left[ 2L+1 \right]^{-1} <J_f j_f L_f S_f| \vec{\sigma}_q \vec{Y}_{L, J}^{\lambda} |J_i j_i L_i S_i> = \nonumber \\
=\sqrt{\frac{2L_i + 1}{4 \pi (2L_f + 1)}} <L_i 0 L 0 | L_f 0> \sum_{s, s_q, s_{\overline q}} (2 s_q \delta_{s, 0} - \sqrt{2} \delta_{s \neq 0}) \times \nonumber \\
    \times <L (\lambda - s) 1 s | J \lambda> \times \nonumber \\
    \times <L_f (j_f - (s_q + s_{\overline{q}}+s)) S_f (s_q + s_{\overline{q}}+s) | J_f j_f> \times \nonumber \\
    \times <L_i (j_i - (s_q + s_{\overline{q}})) S_i (s_q + s_{\overline{q}}) | J_i j_i> \times \nonumber \\
    \times <L_i (j_i - (s_q + s_{\overline{q}})) L (\lambda - s) | L_f (j_f - (s_q + s_{\overline{q}} + s))> \times \nonumber \\
    \times <\frac{1}{2} (s_q+s) \frac{1}{2} s_{\overline{q}} | S_f (s_q + s_{\overline{q}} + s)> \times \nonumber \\
    \times <\frac{1}{2} s_q \frac{1}{2} s_{\overline{q}} | S_i (s_q + s_{\overline{q}})>
$$

In [552]:
def coefC(ins, outs, L, J, lam):
    res = 0
    for sq in (-1/2, 1/2):
        for sqbar in (-1/2, 1/2):
            for s in (-1, 0, 1):
                buf = (2*sq if s == 0 else -sp.sqrt(2)*s)\
                     * clebsch(L, lam - s, 1, s, J, lam)\
                     * clebsch(outs["L"], outs["j"] - (sq+sqbar+s), outs["S"], sq+sqbar+s, outs["J"], outs["j"])\
                     * clebsch(ins["L"], ins["j"] - (sq+sqbar), ins["S"], sq+sqbar, ins["J"], ins["j"])\
                     * clebsch(ins["L"], ins["j"] - (sq+sqbar), L, lam - s, outs["L"], outs["j"] - (sq+sqbar+s))\
                     * clebsch(1/2, sq+s, 1/2, sqbar, outs["S"], sq+sqbar+s)\
                     * clebsch(1/2, sq, 1/2, sqbar, ins["S"], sq+sqbar)
                res += buf
    res *= sp.sqrt((2*ins["L"]+1)/4/sp.pi/(2*outs["L"]+1))*clebsch(ins["L"], 0, L, 0, outs["L"], 0)
    return res

$$ Q_{J}^\lambda = \left[r \sqrt{2L+1} <L 0 1 0 | J 0>\right]^{-1} <J_f j_f L_f S_f| \vec{r} \vec{Y}_{L J}^{\lambda} |J_i j_i L_i S_i> = \nonumber \\
= \delta_{S_i, S_f} \sqrt{\frac{(2L_i + 1)}{(4 \pi (2L_f+1))}} <J 0 L_i 0 | L_f 0>  \times \nonumber \\
\times \sum_{s} <L_f (j_f - s) S_f s | J_f j_f> \times \nonumber \\
\times <L_i (j_i - s) S_i s | J_i j_i> \times \nonumber \\
\times <J \lambda L_i (j_i - s) | L_f (j_f - s)>
$$

In [553]:
def coefQ(ins, outs, J, lam):
    if round(2*ins["S"]+1) != round(2*outs["S"]+1):
        return 0
    res = 0
    for s in range(-ins["S"], ins["S"]+1):
        buf = clebsch(outs["L"], outs["j"] - s, outs["S"], s, outs["J"], outs["j"])\
            * clebsch(ins["L"], ins["j"] - s, ins["S"], s, ins["J"], ins["j"])\
            * clebsch(J, lam, ins["L"], ins["j"]-s, outs["L"], outs["j"]-s)
        res += buf
    res *= sp.sqrt((2*ins["L"]+1)/4/sp.pi/(2*outs["L"]+1))*clebsch(J, 0, ins["L"], 0, outs["L"], 0)
    return res

$$ H_{int} = \sum_{J} H_{int}^{EJ} + H_{int}^{MJ} $$

$$ H_{int}^{EJ} = 2 \sqrt{2 \pi} \mathrm{i}^{J} \delta_{2 \nmid J} (E_f - E_i) (2J+1) \sqrt{J(J+1)} Q_{J}^{-\lambda} \frac{j_J(\frac{kr}{2})}{k} - \color{red}{2}\frac{\lambda k}{2 \mu} \delta_{2 \nmid S_i + S_f + J} \sqrt{2 \pi} \mathrm{i}^{J+1} \left( \sqrt{(J+1)(2J-1)}j_{J-1}(\frac{kr}{2}) C_{J-1, J}^{-\lambda} - \sqrt{J(2J+3)}j_{J+1}(\frac{kr}{2})C_{J+1, J}^{-\lambda} \right) $$

$$ H_{int}^{MJ} = -\sqrt{2 \pi} \mathrm{i}^{J} (2J+1) \frac{k}{2 \mu} j_{J}(\frac{kr}{2}) \delta_{2 \mid S_i + S_f + J} C_{J, J}^{-\lambda} $$

Notice $2$ which has been restored from asymptotics. No description by now

In [596]:
def Hint(ins, outs, k, r, lam):
    mu = 1
    res = 0
    for J in range(1,15):
        buf = 2*(0 if J % 2 == 0 else 1)*(outs["E"]-ins["E"])*sp.sqrt(J*(J+1))*coefQ(ins, outs, J, -lam)/k\
            + 2*k/2*(0 if J+ins["S"]+outs["S"] % 2 == 0 else 1)*coefC(ins, outs, J, J, -lam)
        buf *= sp.sqrt(2*sp.pi)*(1j)**J*(2*J+1)/mu*spherical_jn(J, k*r/2)
        res += buf
        
        buf = -sp.sqrt(2*sp.pi)*(1j)**(J+1)*lam*k/2/mu*(1 if outs["S"]+ins["S"]+J % 2 == 0 else 0)*(sp.sqrt((J+1)*(2*J-1))*spherical_jn(J-1, k*r/2)*coefC(ins,outs,J-1,J,-lam)-sp.sqrt(J*(2*J+3))*spherical_jn(J+1, k*r/2)*coefC(ins,outs,J+1,J,-lam))
        res += buf
    return res    

In [594]:
Hint_raw({"J": 1, "j": -1, "L": 1, "S": 0, "E": 1}, {"J": 0, "j": 0, "L": 0, "S": 0, "E":0}, 12, 1, -1)

(2.5851742576609877e-13-0.04843677852685696j)

In [595]:
Hint({"J": 1, "j": -1, "L": 1, "S": 0, "E": 1}, {"J": 0, "j": 0, "L": 0, "S": 0, "E":0}, 12, 1, -1)

-0.04843677678759261j

## Test coefQ

$$
    <J_f j_f L_f S_f|\vec{r}\vec{\epsilon}^\star_\lambda \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}}|J_i j_i L_i S_i> = \sqrt{2 \pi} \sum_{J = 1}^{\infty} \mathrm{i}^J \sqrt{2J+1} \left[ \color{gray}{-\lambda j_J(\frac{kr}{2}) r Q_{J}^{-\lambda} \sqrt{2J+1} <J 0 1 0 | J 0>} + \nonumber \\
    + \mathrm{i} \left( \sqrt{\frac{J+1}{2J+1}} r \sqrt{2J-1} <(J-1) 0 1 0 | J 0> j_{J-1}(\frac{kr}{2}) Q_{J}^{-\lambda} - \sqrt{\frac{J}{2J+1}}r\sqrt{2J+3}<(J+1)010|J0>j_{J+1}(\frac{kr}{2})Q_{J}^{-\lambda} \right) \right]
$$

In [535]:
def spinlessAv_Q(ins, outs, k, r, lam):
    raw = spinlessAv(ins, outs, k, r, lam)
    res = 0
    for J in range(1, 11):
        buf = -lam*spherical_jn(J, k*r/2)*r*coefQ(ins, outs, J, -lam)*sp.sqrt(2*J+1)*clebsch(J, 0, 1, 0, J, 0)*0 +\
              +1j*(sp.sqrt((J+1)/(2*J+1))*r*sp.sqrt(2*J-1)*clebsch(J-1,0,1,0,J,0)*spherical_jn(J-1, k*r/2)*coefQ(ins, outs, J, -lam) -\
                   sp.sqrt((J)/(2*J+1))*r*sp.sqrt(2*J+3)*clebsch(J+1,0,1,0,J,0)*spherical_jn(J+1, k*r/2)*coefQ(ins,outs,J,-lam))
        buf *= 1j**J*sp.sqrt(2*J + 1)
        res += buf
    res *= sp.sqrt(2*sp.pi)
    return (res, raw, res-raw)

In [575]:
spinlessAv_Q({"J": 1, "j": 1, "L": 1, "S":1, "E": 1}, {"J": 1, "j": 1, "L": 0, "S": 1, "E":0}, 0.3, 1, -1)

(0j,
 (-8.479828283150676e-15+1.4173672719092547e-14j),
 (8.479828283150676e-15-1.4173672719092547e-14j))

$$
    <J_f j_f L_f S_f|\vec{\sigma} \vec{\epsilon}^\star_\lambda \mathrm{e}^{\mathrm{i} k \frac{r}{2} \cos{\theta}}|J_i j_i L_i S_i> = \sqrt{2 \pi} \sum_{J = 1}^{\infty} \mathrm{i}^J \sqrt{2J+1} \left[ -\lambda j_J(\frac{kr}{2}) \sqrt{2J+1} C_{J,J}^{-\lambda} + \nonumber \\
    + \mathrm{i} \left( \sqrt{\frac{J+1}{2J+1}} \sqrt{2J-1} j_{J-1}(\frac{kr}{2}) C_{J-1,J}^{-\lambda} - \sqrt{\frac{J}{2J+1}} \sqrt{2J+3} j_{J+1}(\frac{kr}{2}) C_{J+1,J}^{-\lambda} \right) \right]
$$

In [560]:
def spinAv_C(ins, outs, k, r, lam):
    raw = spinAv(ins, outs, k, r, lam)
    res = 0
    for J in range(1, 10):
        buf = -lam*spherical_jn(J, k*r/2)*sp.sqrt(2*J+1)*coefC(ins, outs, J, J, -lam) +\
              +1j*(sp.sqrt((J+1)/(2*J+1))*spherical_jn(J-1, k*r/2)*sp.sqrt(2*J-1)*coefC(ins, outs, J-1, J, -lam) -\
                   sp.sqrt((J)/(2*J+1))*spherical_jn(J+1, k*r/2)*sp.sqrt(2*J+3)*coefC(ins,outs, J+1, J,-lam))
        buf *= 1j**J*sp.sqrt(2*J + 1)
        res += buf
    res *= sp.sqrt(2*sp.pi)
    return (res, raw, res-raw)

In [570]:
spinAv_C({"J": 1, "j": 1, "L": 1, "S": 1, "E": 1}, {"J": 1, "j": 0, "L": 0, "S": 1, "E":0}, 10, 1, 1)

(-0.11646025470871753j,
 (-1.0610389811020397e-16-0.11646025792685778j),
 (1.0610389811020397e-16+3.218140254279689e-09j))